In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [101]:
def clean_models(df):
    df = df.rename({'Training power (Watts) ':'Training power (Watts)', 'Training energy (kWh) ':'Training energy (kWh)'}, axis=1)
    df = df.drop(['Base model', 'Finetune compute (FLOP)', 'Finetune compute notes'], axis=1)
    df['Domain'] = df['Domain'].astype("string").str.split(",").map(lambda x: sorted(x)).map(lambda x: ','.join(x))
    df['System'] = df['System'].astype("string")
    df['Task'] = df['Task'].astype("string").str.split(',').map(lambda x: sorted(x)).map(lambda x: ','.join(x))
    df['Model accessibility'] = df['Model accessibility'].astype('category')
    df['Citations'] = df['Citations'].fillna(0).astype('uint32')
    df['Publication date'] = pd.to_datetime(df['Publication date'])
    df['Organization'] = df['Organization'].astype('category')

    df['Organization categorization (from Organization)'] = df['Organization categorization (from Organization)']\
                                                              .astype('string')\
                                                              .str.strip()\
                                                              .str.split(',')\
                                                              .apply(lambda x: ','.join(sorted(list(set(x)))))
    return df

def clean_bloom(df):
    df = df.drop(['cloud_provider', 'cloud_region'], axis=1)
    df = df[~df.drop(['timestamp', 'project_name', 'run_id'], axis=1).isna().any(axis=1)]
    df.loc[312, 'timestamp'] = '2022-05-18T12:04:08'
    df = df.drop([767])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

models_df = pd.read_csv("large_scale_ai_models_added_cols.csv")
bloom_df = pd.read_csv("bloom_emissions.csv")
models_df = clean_models(models_df)
bloom_df = clean_bloom(bloom_df)
models_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 30 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   System                                           223 non-null    string        
 1   Domain                                           223 non-null    object        
 2   Task                                             223 non-null    object        
 3   Authors                                          132 non-null    object        
 4   Model accessibility                              184 non-null    category      
 5   Link                                             220 non-null    object        
 6   Citations                                        223 non-null    uint32        
 7   Reference                                        185 non-null    object        
 8   Publication date                        

In [103]:
# save the clean dataframes
models_df.to_csv('models_clean.csv', index=False)
bloom_df.to_csv('bloom_clean.csv', index=False)

In [99]:
models_df['Organization categorization (from Organization)']\
         .astype('string')\
         .str.strip()\
         .str.split(',')\
         .apply(lambda x: ','.join(sorted(list(set(x)))))

0      Industry
1      Industry
2      Industry
3      Industry
4      Industry
         ...   
218    Industry
219    Industry
220    Industry
221    Industry
222    Industry
Name: Organization categorization (from Organization), Length: 223, dtype: object

In [45]:
#good columns for statistical analysis in models

models_cols = [
    'System',
    'Domain',
    'Task',
    'Parameters',
    'Training compute (FLOP)',
    'Training dataset size (datapoints)',
    'Training time (hours)',
    'Training hardware',
    'Hardware quantity',
    'Training power (Watts)',
    'Training energy (kWh)',
    'Carbon Emissions from training (lb)'
]

models_good_df = models_df[models_cols]
models_good_df.to_csv('models_clean_analysis.csv', index=False)

KeyError: "None of [Index(['System', 'Domain', 'Task', 'Parameters', 'Training compute (FLOP)',\n       'Training dataset size (datapoints)', 'Training time (hours)',\n       'Training hardware', 'Hardware quantity', 'Training power (Watts)',\n       'Training energy (kWh)', 'Carbon Emissions from training (lb)'],\n      dtype='object')] are in the [index]"